In [2]:
import pandas as pd
import pyprind
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.io as pio
pio.renderers.default = 'vscode'

In [3]:
file_path = "C:/Users/ryan_/OneDrive/Desktop/Suicide Watch Detection/src/Data/cleaned_data.csv"

# Define column names based on the first line of the file
df = pd.read_csv(file_path)



df.rename(columns={'class': 'label'}, inplace=True)


df.head()

,text,label
0,"['wife', 'threaten', 'suicide', 'recently', 'l...",1
1,"['weird', 'get', 'affect', 'come', 'someone', ...",0
2,"['finally', 'almost', 'never', 'hear', 'bad', ...",0
3,"['need', 'help', 'just', 'help', 'cry', 'hard']",1
4,"['lose', 'hello', 'name', 'struggle', 'year', ...",1


## RNN


#### Creating datasets

In [133]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2, random_state=10)


train_data, valid_data = train_test_split(train_data, test_size=0.1, random_state=10)

train_data['text'] = train_data['text'].apply(eval)
test_data['text'] = test_data['text'].apply(eval)
valid_data['text'] = valid_data['text'].apply(eval)



                                                     text  label
135094         ['hey', 'girl', 'tree', 'hear', 'whisper']      0
10717   ['girl', 'gon', 'need', 'help', 'mean', 'steal...      0
178488  ['want', 'make', 'only', 'fan', 'next', 'year'...      0
52509   ['even', 'brush', 'tooth', 'get', 'bed', 'with...      1
172712  ['dump', 'well', 'pretty', 'upset', 'relations...      0
text     object
label     int64
dtype: object


In [135]:
from collections import Counter, OrderedDict
from torchtext.vocab import vocab

# Initialize token_counts Counter
token_counts = Counter()

# Convert train_dataset to DataFrame
def tokenizer(text):
    # Tokenize the text
    tokenized = text
    tokenized = [token.strip("[]',") for token in tokenized]
    
    # Update token_counts with tokenized text
    token_counts.update(tokenized)
    
    # Replace unknown tokens with <unk>
    tokenized = [token if token_counts[token] > 0 else "<unk>" for token in tokenized]
    
    return tokenized

# Iterate over train_data['text'] and update token_counts
for text in train_data['text']:
    tokenizer(text)

# Sort token counts by frequency
sorted_by_freq_tuples = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)

# Convert to OrderedDict and create vocab object
ordered_dict = OrderedDict(sorted_by_freq_tuples)
vocab_object = vocab(ordered_dict)

# Insert special tokens
vocab_object.insert_token("<pad>", 0)
vocab_object.insert_token("<unk>", 1)

# Set default index
vocab_object.set_default_index(0)

# Print vocabulary size
print('Vocab Size:', len(vocab_object))

# Print vocabulary object
print(vocab_object)


Vocab Size: 23014
Vocab()


In [136]:
# Print token indices for given tokens, handling OOV tokens
token_indices = [vocab_object[token] if token in vocab_object else vocab_object['<unk>'] for token in ['lok', 'fish', 'help', 'test']]
print(token_indices)

[1, 1983, 24, 463]


# THIS IS THE PROBLEM!!!

In [179]:
def text_pipeline(tokens):
    # Convert the list of tokens to tensor indices using vocab_object
    token_indices = [vocab_object[token] if token in vocab_object else vocab_object['<unk>'] for token in tokens]
    return token_indices

label_pipeline = lambda x: 1. if x == 1 else 0.

# Debugging the text_pipeline
print("Testing text_pipeline:")
tokens = ['this', 'is', 'a', 'test']
print("Input tokens:", tokens)
processed_tokens = text_pipeline(tokens)
print("Processed tokens:", processed_tokens)

# Debugging the label_pipeline
print("\nTesting label_pipeline:")
label = 1
print("Input label:", label)
processed_label = label_pipeline(label)
print("Processed label:", processed_label)


    


Testing text_pipeline:
Input tokens: ['this', 'is', 'a', 'test']
Processed tokens: [928, 1, 1, 463]

Testing label_pipeline:
Input label: 1
Processed label: 1.0


# TASKS FOR Today:
### Build 3 RNN Models using these three types of recurrent layers
#### RNN
#### LSTM
#### GRU


In [142]:
# Check the structure and contents of the DataFrame
print(train_data['text'].head())  # Display the first few rows of the 'text' column
print(train_data['text'].dtype)   # Check the data type of the 'text' column


print(len(vocab_object))  # Check the size of the vocabulary
print(vocab_object.get_itos())  # Display the first few tokens in the vocabulary


135094                     [hey, girl, tree, hear, whisper]
10717     [girl, gon, need, help, mean, steal, period, f...
178488    [want, make, only, fan, next, year, realize, w...
52509     [even, brush, tooth, get, bed, without, put, s...
172712    [dump, well, pretty, upset, relationship, defi...
Name: text, dtype: object
object
23014
['<pad>', '<unk>', 'get', 'like', 'want', 'feel', 'know', 'life', 'think', 'would', 'make', 'go', 'people', 'one', 'friend', 'even', 'year', 'really', 'say', 'day', 'thing', 'try', 'tell', 'well', 'help', 'never', 'good', 'see', 'take', 'live', 'much', 'filler', 'talk', 'could', 'die', 'bad', 'end', 'love', 'kill', 'need', 'work', 'way', 'school', 'someone', 'anything', 'back', 'start', 'still', 'everything', 'something', 'find', 'every', 'family', 'give', 'hate', 'always', 'care', 'nothing', 'keep', 'come', 'anyone', 'leave', 'suicide', 'long', 'right', 'look', 'ever', 'last', 'everyone', 'post', 'stop', 'parent', 'person', 'since', 'ame', 'month', 'po

In [199]:
from torch.utils.data import DataLoader
def collate_batch(batch):
    text_list, label_list, length = [], [], []
    for text, label in batch:
        label_list.append(label_pipeline(label))
        processed_text = torch.tensor(text_pipeline(text), dtype=torch.int64)
        text_list.append(processed_text)
        length.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.float32)
    length = torch.tensor(length)
    padded_text_list = nn.utils.rnn.pad_sequence(text_list, batch_first=True)
    return padded_text_list, label_list, length

# Assuming train_data is properly defined
train_dataloader = DataLoader(train_data, batch_size=4, shuffle=True, collate_fn=collate_batch)

In [198]:
batch_size = 32
train_dl = DataLoader(train_data, batch_size=batch_size,shuffle=True,collate_fn=collate_batch)
valid_dl = DataLoader(valid_data, batch_size=batch_size,shuffle=False,collate_fn=collate_batch)
test_dl = DataLoader(test_data, batch_size=batch_size,shuffle=False,collate_fn=collate_batch)



In [182]:
embedding = nn.Embedding(num_embeddings= len(token_counts), embedding_dim=100, padding_idx=0)

In [184]:

import numpy as np
import torch

class SGD:
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate
        
    def step(self, rnn_model, gradients):
        for param_name, gradient in gradients.items():
            setattr(rnn_model, param_name, getattr(rnn_model, param_name) - self.learning_rate * gradient)


class RNN:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Random Initial Weights
        self.Wx = np.random.randn(hidden_size, input_size) * 0.01
        self.Wxy = np.random.randn(hidden_size, hidden_size) * 0.01
        self.Wyz = np.random.randn(output_size, hidden_size) * 0.01
        self.Bh = np.zeros((hidden_size, 1))
        self.Bo = np.zeros((output_size, 1))
        
        self.learning_rate = learning_rate
        self.optimizer = None

    def forward(self, inputs, h_prev):
        self.a = np.dot(self.Wx, inputs) + np.dot(self.Wxy, h_prev) + self.Bh
        self.h = np.tanh(self.a)
        self.y = np.dot(self.Wyz, self.h) + self.Bo
        return self.y, self.h

    def backward(self, d_y, h, h_prev, inputs):
        d_Wyz = np.dot(d_y, self.h.T)
        d_Bo = d_y
        d_h = np.dot(self.Wyz.T, d_y)
        d_a = np.dot(1 - self.h ** 2, d_h)
        d_Bh = d_a
        d_Wxy = np.dot(d_a, h_prev.T)
        d_Wx = np.dot(d_a, inputs.T)
        d_h_prev = np.dot(self.Wxy.T, d_a)
        
        gradients = {
            'd_Wx': d_Wx,
            'd_Wxy': d_Wxy,
            'd_Wyz': d_Wyz,
            'd_Bh': d_Bh,
            'd_Bo': d_Bo,
            'd_h_prev': d_h_prev
        }
        return gradients
    
    def train_step(self, inputs, targets, h_prev):
        # Forward pass
        y_pred, hidden_state = self.forward(inputs, h_prev)

        # Calculate loss
        loss = np.mean((y_pred - targets)**2)

        # Backward pass
        gradients = self.backward(2 * (y_pred - targets), hidden_state, h_prev, inputs)

        # Update weights
        self.optimizer.step(self, gradients)

        return loss

# Example usage:
input_size = 10
hidden_size = 64
output_size = 2
learning_rate = 0.01

rnn_model = RNN(input_size, hidden_size, output_size, learning_rate)




In [185]:
input_size = 10
hidden_size = 64
output_size = 1
learning_rate = 0.01

rnn_model = RNN(input_size, hidden_size, output_size, learning_rate)
rnn_model.optimizer = SGD(learning_rate)

def train_model(dataloader):  # Renamed the function to avoid conflict
    total_acc, total_loss = 0.0, 0.0
    for text_batch, label_batch in dataloader:
        h_prev = np.zeros((hidden_size, text_batch.shape[0]))  # Initialize hidden state
        
        loss = rnn_model.train_step(text_batch, label_batch, h_prev)
        
        total_loss += loss * text_batch.shape[0]
        
        pred = rnn_model.forward(text_batch, h_prev)[0]
        total_acc += ((pred >= 0.5).astype(float) == label_batch).astype(float).sum().item()
        
    return total_acc / len(dataloader.dataset), total_loss / len(dataloader.dataset)

In [186]:
rnn_model = RNN(input_size = 100, hidden_size = 64, output_size = 2)
learning_rate = 0.01
num_epochs = 10




In [187]:
def evaluate(dataloader):
    total_acc, total_loss = 0, 0
    with torch.no_grad():
        for text_batch, label_batch in dataloader:
            h_prev = np.zeros((hidden_size, text_batch.shape[0]))  # Initialize hidden state
            
            loss = rnn_model.train_step(text_batch, label_batch, h_prev)
            
            total_loss += loss * text_batch.shape[0]
            
            pred = rnn_model.forward(text_batch, h_prev)[0]
            total_acc += ((pred >= 0.5).astype(float) == label_batch).astype(float).sum().item()
        
    return total_acc / len(dataloader.dataset), total_loss / len(dataloader.dataset)

In [188]:
torch.manual_seed(1)
for epoch in range(num_epochs):
    acc_train, loss_train = train_model(train_dl)
    acc_valid, loss_valid = evaluate(valid_dl)
    print(f'Epoch {epoch} accuracy: {acc_train:.4f}'
          f' val_accuracy: {acc_valid:.4f}')

KeyError: 129376